In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/whatsapp-chat-analyzer-dataset/WhatsApp Chat with ECE A 5th semester.txt
/kaggle/input/whatsapp-chat-analyzer-dataset/WhatsApp Chat with HackNx Community.txt


In [3]:
import re

In [4]:
f = open('/kaggle/input/whatsapp-chat-analyzer-dataset/WhatsApp Chat with ECE A 5th semester.txt','r',encoding='utf-8')

In [61]:
# data = f.read()

In [5]:
print(type(data))

<class 'str'>


In [6]:
# Step 1: Read text file
with open('/kaggle/input/whatsapp-chat-analyzer-dataset/WhatsApp Chat with ECE A 5th semester.txt','r',encoding='utf-8') as f:
    data = f.read()

# Step 2: Regular expression pattern to extract datetime and message
pattern = r'(\d{1,2}/\d{1,2}/\d{2}), (\d{1,2}:\d{2}\s?[AP]M) - (.*?)(?=(?:\n\d{1,2}/\d{1,2}/\d{2}, \d{1,2}:\d{2}\s?[AP]M -)|\Z)'

# Step 3: Find all messages
matches = re.findall(pattern, data, re.DOTALL)

# Step 4: Store extracted messages
message_list = []

for date, time, message in matches:
    datetime_str = f"{date}, {time}"
    message_cleaned = message.strip()
    
    # Print output
    print(f" DateTime: {datetime_str}")
    print(f" Message: {message_cleaned}")
    print("-" * 60)
    
    # Add to list for DataFrame
    message_list.append({
        "datetime": datetime_str,
        "message": message_cleaned
    })

# Step 5: Convert to DataFrame
df = pd.DataFrame(message_list)

# Step 6: Optional – convert to datetime object
df["datetime"] = pd.to_datetime(df["datetime"], format="%m/%d/%y, %I:%M %p")

# Step 7: Show DataFrame
print("\n Final DataFrame:")
print(df.head())


 DateTime: 3/29/24, 3:59 PM
 Message: Messages and calls are end-to-end encrypted. Only people in this chat can read, listen to, or share them. Learn more.
------------------------------------------------------------
 DateTime: 9/20/23, 11:55 AM
 Message: +91 93148 81683 created group "B.Tech Ist year ECE-A"
------------------------------------------------------------
 DateTime: 3/29/24, 3:59 PM
 Message: Love Kishore Sir "Mechanical" added you
------------------------------------------------------------
 DateTime: 3/30/24, 10:02 AM
 Message: +91 96723 53253: All the students are hereby informed that the mid term examination is about to start. Any student who is still not present is required to report immediately in class.
------------------------------------------------------------
 DateTime: 3/30/24, 10:43 AM
 Message: +91 98281 97248: Following students not present in second class

1. Gaurav
2. Jangid vinaykumar
3. Jitendra jat
4. Krish kumawat                    5 lakshya singh
---

In [7]:
df

datetime                                            message
0    2024-03-29 15:59:00  Messages and calls are end-to-end encrypted. O...
1    2023-09-20 11:55:00  +91 93148 81683 created group "B.Tech Ist year...
2    2024-03-29 15:59:00            Love Kishore Sir "Mechanical" added you
3    2024-03-30 10:02:00  +91 96723 53253: All the students are hereby i...
4    2024-03-30 10:43:00  +91 98281 97248: Following students not presen...
...                  ...                                                ...
1838 2025-07-04 11:26:00  +91 95294 06677: *Announcement of the MIC Driv...
1839 2025-07-04 11:26:00  ~ 😊 changed this group's settings to allow all...
1840 2025-07-04 11:28:00  +91 95294 06677: Share screenshot watching ses...
1841 2025-07-04 11:44:00                           +91 95294 06677: Example
1842 2025-07-04 11:58:00             Vinita Ma'am: This message was deleted

[1843 rows x 2 columns]

In [8]:
# Function to split name and message if ":" exists
def split_sender_message(msg):
    if ": " in msg:
        name, text = msg.split(": ", 1)
        return name.strip(), text.strip()
    else:
        # System message or deleted message
        return np.nan, msg.strip()

# Apply function to each row
df[["name", "text"]] = df["message"].apply(lambda x: pd.Series(split_sender_message(x)))

# Show results
# print(df[["datetime", "name", "text"]].head())


In [10]:
df = df.fillna('unknown')

In [14]:
df['month'] = df['datetime'].dt.month_name()

In [16]:
df['day'] = df['datetime'].dt.day

In [18]:
df['hour'] = df['datetime'].dt.hour

In [20]:
df['minute'] = df['datetime'].dt.minute

In [21]:
df.head()

datetime                                            message  \
0 2024-03-29 15:59:00  Messages and calls are end-to-end encrypted. O...   
1 2023-09-20 11:55:00  +91 93148 81683 created group "B.Tech Ist year...   
2 2024-03-29 15:59:00            Love Kishore Sir "Mechanical" added you   
3 2024-03-30 10:02:00  +91 96723 53253: All the students are hereby i...   
4 2024-03-30 10:43:00  +91 98281 97248: Following students not presen...   

              name                                               text  \
0          unknown  Messages and calls are end-to-end encrypted. O...   
1          unknown  +91 93148 81683 created group "B.Tech Ist year...   
2          unknown            Love Kishore Sir "Mechanical" added you   
3  +91 96723 53253  All the students are hereby informed that the ...   
4  +91 98281 97248  Following students not present in second class...   

       month  day  hour  minute  
0      March   29    15      59  
1  September   20    11      55  
2      March   29    15      59  
3      March   30    10       2  
4      March   30    10      43

In [24]:
df = df.rename(columns={'name':'users'})